# import packages

In [32]:
%matplotlib inline 
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy.stats import ttest_ind

In [33]:
path = './fworrrevisedwheeler'

# import approvals data

In [40]:
file_list = ['/DATA - Wheeler (2017 approvals - felony).xlsx',
             '/DATA - Wheeler (2018 approvals - felony).xlsx',
             '/DATA - Wheeler (2017 approvals - misd).xlsx',
             '/DATA - Wheeler (2018 approvals - misd).xlsx']
df_big_list = []
for file in file_list: 
    df_list = []
    for i in range(5):
        df_iter = pd.read_excel(path+file, sheet_name=i)
        df_list.append(df_iter)
    approvals = pd.concat(df_list)
    approvals['CaseNumber'] = approvals['CaseNumber'].astype(str)
    approvals['charge_type'] = np.where('misd' in file, 'misd', 'felony')
    df_big_list.append(approvals)
approvals = pd.concat(df_big_list)

In [42]:
approvals['charge_type'].value_counts()

misd      84714
felony    38446
Name: charge_type, dtype: int64

# import forfeitures data

In [43]:
file_list = ['/DATA - Wheeler (2017 forfeitures - felony).xlsx',
             '/DATA - Wheeler (2018 forfeitures - felony).xlsx',
             '/DATA - Wheeler (2017 forfeitures - misd).xlsx',
             '/DATA - Wheeler (2018 forfeitures - misd).xlsx']
df_big_list = []
for file in file_list: 
    df_list = []
    for i in range(5):
        df_iter = pd.read_excel(path+file, sheet_name=i)
        df_list.append(df_iter)
        forfeitures = pd.concat(df_list)
    forfeitures['CaseNumber'] = forfeitures['CaseNumber'].str[:-1]
    forfeitures['CaseNumber'] = forfeitures['CaseNumber'] + '0'
    forfeitures['ForfDate'] = forfeitures['ForfDate'].astype(int)
    forfeitures['ForfDate'] = pd.to_datetime(forfeitures['ForfDate'], format='%Y%m%d')
    forfeitures['charge_type'] = np.where('misd' in file, 'misd', 'felony')
    df_big_list.append(forfeitures)
forfeitures = pd.concat(df_big_list)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [44]:
forfeitures['charge_type'].value_counts()

misd      20673
felony     5367
Name: charge_type, dtype: int64

# determine who forfeited

In [71]:
df = approvals.merge(forfeitures, on=['charge_type', 'CaseNumber'], how='left', suffixes=('_app', '_for'))
df = df.dropna(subset=['BondAmtMade'])
df['forfeit'] = np.where(df['ForfDate'] > df['DateApproved'], 1, 0)

# bin bond amount

In [72]:
df['bond_group'] = pd.cut(df['BondAmtMade'], 
       bins=[0, 2000, 2001, 5000, 10000, 20000, 20001, 500001], 
       right=False, 
       labels=['< $2,000',
              '$2,000',
              '$2,001-$4,999',
              '$5,000-$9,999',
              '$10,000-$19,999',
              '$20,000',
              '> $20,000'])

# calc summary stats

In [73]:
df_mean = df.groupby(['charge_type', 'CodeDesc', 'bond_group', 'forfeit']).agg({'BondAmtMade': 'mean'}).reset_index()
df_mean = df_mean.pivot_table(index=['charge_type', 'CodeDesc', 'bond_group'], columns='forfeit', values='BondAmtMade')
df_mean.columns=['No Forfeit', 'Forfeit']

# run t-test

In [145]:
df = df.dropna(subset=['charge_type', 'CodeDesc', 'bond_group'])
df = df.set_index(['charge_type', 'CodeDesc', 'bond_group'])
df_list = []
for x in df.index.unique():
    df_bin = df.copy(deep=True)
    df_bin = df_bin.loc[x]
    bond_forf = df_bin.loc[df_bin['forfeit']==1]['BondAmtMade']
    bond_no_forf = df_bin.loc[df_bin['forfeit']==0]['BondAmtMade']
    t, p = ttest_ind(bond_forf, bond_no_forf, equal_var=False)
    d = {'t-stat': t, 'p-value': p}    
    tr = pd.DataFrame(data=d, index=[0])
    df_list.append(tr)
tr = pd.concat(df_list)
tr = tr.set_index(df.index.unique())
df_out = df_mean.merge(tr, left_index=True, right_index=True)
df_out['t-stat'] = df_out['t-stat'].fillna(0)
df_out['p-value'] = df_out['p-value'].fillna(1)

KeyError: ['charge_type', 'CodeDesc', 'bond_group']

# count the number of cases

In [155]:
df_count = df.groupby(['charge_type', 'CodeDesc', 'bond_group']).agg({'CaseNumber': 'count'})
df_out = df_out.merge(df_count, left_index=True, right_index=True)
df_out = df_out.rename(columns={'CaseNumber': 'N Cases'})

# count the number of forfeitures

In [172]:
df_out = df_out.merge(df.groupby(['charge_type', 'CodeDesc', 'bond_group']).agg({'forfeit': 'sum'}),
                     left_index=True, right_index=True)
df_out = df_out.rename(columns={'forfeit': 'N Forfeitures'})

# calculate the difference in mean bond rates

In [179]:
df_out['Difference'] = df_out['Forfeit'] - df_out['No Forfeit']
df_out['Relative Difference'] = (df_out['Forfeit'] - df_out['No Forfeit']) / df_out['No Forfeit']

# write to excel

In [182]:
df_out = df_out.reset_index()

In [184]:
df_out.to_excel('bond_forfeit.xlsx', index=False)

In [186]:
df.to_pickle('df_bond_forfeit.pkl')

# type of bond by month

In [262]:
# felony

df = pd.read_pickle('df_bond_forfeit.pkl')

df = df.reset_index()

df = df[df['charge_type']=='felony']

df = df[['charge_type', 'CodeDesc', 'DateApproved', 'CaseNumber']]

df = df.drop_duplicates()

df = df.dropna()

df['month_approved'] = df['DateApproved'].values.astype('datetime64[M]')

df = df.groupby(['charge_type', 'CodeDesc', 'month_approved']).agg({'CaseNumber': 'count'}).reset_index()

df = df.rename(columns={'CaseNumber': 'n_cases'})

df_f = df.copy(deep=True)

In [263]:
# misd

df = pd.read_pickle('df_bond_forfeit.pkl')

df = df.reset_index()

df = df[df['charge_type']=='misd']

df = df[['charge_type', 'CodeDesc', 'DateApproved', 'CaseNumber']]

df = df.drop_duplicates()

df = df.dropna()

df['month_approved'] = df['DateApproved'].values.astype('datetime64[M]')

df = df.groupby(['charge_type', 'CodeDesc', 'month_approved']).agg({'CaseNumber': 'count'}).reset_index()

df = df.rename(columns={'CaseNumber': 'n_cases'})

df_misd = df.copy(deep=True)

In [264]:
df = pd.concat([df_f, df_misd])

In [267]:
df.to_excel('bond_time.xlsx')